In [1]:
import boto3
import botocore

import json

In [2]:
# AWS Clients

braket_config = botocore.client.Config(user_agent_extra="BraketSchemas/1.8.0")

aws_braket = boto3.client("braket", config=braket_config)
aws_s3 = boto3.client("s3")

In [3]:
# Load info for Tasks simulated on SV1

SV1_ARN = 'arn:aws:braket:::device/quantum-simulator/amazon/sv1'

response = aws_braket.search_quantum_tasks(filters=[{
    'name': 'deviceArn',
    'operator': 'EQUAL',
    'values': [SV1_ARN]
}], maxResults=25)

In [4]:
quantum_tasks = response['quantumTasks']

quantum_task = quantum_tasks[0]

quantum_task

{'createdAt': datetime.datetime(2023, 2, 28, 16, 55, 2, 235000, tzinfo=tzutc()),
 'deviceArn': 'arn:aws:braket:::device/quantum-simulator/amazon/sv1',
 'endedAt': datetime.datetime(2023, 2, 28, 16, 55, 11, 962000, tzinfo=tzutc()),
 'outputS3Bucket': 'amazon-braket-us-east-1-196332284574',
 'outputS3Directory': 'tasks/9581aea9-5357-4dba-bf3e-f97e19d5750e',
 'quantumTaskArn': 'arn:aws:braket:us-east-1:196332284574:quantum-task/9581aea9-5357-4dba-bf3e-f97e19d5750e',
 'shots': 1000,
 'status': 'COMPLETED',
 'tags': {}}

In [5]:
s3_bucket_name = quantum_task['outputS3Bucket']
s3_directory = quantum_task['outputS3Directory']
    
s3_bucket_name, s3_directory

('amazon-braket-us-east-1-196332284574',
 'tasks/9581aea9-5357-4dba-bf3e-f97e19d5750e')

In [6]:
# Download JSON

RESULTS_JSON_FILENAME = 'results-downloaded.json'

aws_s3.download_file(Bucket=s3_bucket_name,
                     Key=s3_directory + '/results.json',
                     Filename=RESULTS_JSON_FILENAME)

In [7]:
with open(RESULTS_JSON_FILENAME) as json_file:
    
    result_dict = json.load(json_file)    

In [8]:
# result_dict['measurements'][0]

### QASM

In [11]:
from qiskit import qasm3

In [12]:
# Write QASM to File

QASM_FILENAME = "qasm_file_1.qasm"

TRANSLATIONS = {"cnot": "cx",
                "#pragma": "// #pragma"}


qasm_string = result_dict['additionalMetadata']['action']['source']

for translation in TRANSLATIONS.items():
    
    qasm_string = qasm_string.replace(*translation)

qasm_lines = qasm_string.splitlines()

qasm_lines.insert(1, 'include "stdgates.inc";')

with open(QASM_FILENAME, "w") as qasm_file:
    
    qasm_file.writelines(line + '\n' for line in qasm_lines)

In [13]:
circuit = qasm3.load(QASM_FILENAME)

circuit.draw(fold=200)

┌───────┐                                                 ┌───────────┐┌────────────┐                                                                          
 q_0: ┤ Ry(0) ├───────■──────────────────────────────────────■──┤ Ry(4.093) ├┤ Ry(2.4073) ├──────────────────────────────────────────────────────────────────────────
      ├───────┤       │                                      │  └───┬───┬───┘└────────────┘                                                                          
 q_1: ┤ Ry(0) ├───────┼──────────────────────────────────────┼──────┤ X ├────────────────────────────────────────────────────────────────────────────────────────────
      ├───────┤       │                                      │      └─┬─┘                                                                                            
 q_2: ┤ Ry(0) ├───────┼──────────────────────────────────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────
      ├───────┤       │                                      │        │                                                                                              
 q_3: ┤ Ry(0) ├───────┼──────────────────────────────────────┼────────┼──────────────────────────────────────────────────────────────────────────────────────────────
      ├───────┤       │                     ┌─────────────┐  │        │      ┌────────────┐┌─────────────┐                                                           
 q_4: ┤ Ry(0) ├───────┼─────────────■───────┤ Rz(0.11555) ├──┼────────┼──────┤ Ry(5.7447) ├┤ Ry(0.86008) ├───────────────────────────────────────────────────────────
      ├───────┤     ┌─┴─┐           │       └┬───────────┬┘  │        │      ├────────────┤└─────────────┘                                                           
 q_5: ┤ Ry(0) ├─────┤ X ├───────────┼────────┤ Ry(1.711) ├───┼────────┼──────┤ Ry(1.7219) ├──────────────────────────────────────────────────────────────────────────
      ├───────┤     └───┘           │        └───────────┘   │        │      └────────────┘     ┌───┐                                        ┌───┐                   
 q_6: ┤ Ry(0) ├─────────────────────┼────────────────────────┼────────┼─────────────────────────┤ X ├────────────────────────────────────────┤ X ├───────────────────
      ├───────┤ ┌───────────┐       │                        │        │                         └─┬─┘                                        └─┬─┘                   
 q_7: ┤ Ry(0) ├─┤ Ry(3.303) ├───────┼────────────────────────┼────────┼───────────────────────────┼────────────────────────────────────────────┼─────────────────────
      ├───────┤ ├───────────┤       │                        │        │                           │                                            │  ┌───┐              
 q_8: ┤ Ry(0) ├─┤ Ry(5.655) ├───────┼────────────────────────┼────────┼───────────────────────────┼────────────────────────────────────────────┼──┤ X ├──────────────
      ├───────┤┌┴───────────┴┐      │        ┌───────────┐   │        │      ┌───────────┐        │                           ┌────────────┐   │  └─┬─┘              
 q_9: ┤ Ry(0) ├┤ Ry(0.73985) ├──────┼────────┤ Rz(3.482) ├───┼────────┼──────┤ Rx(1.297) ├────────┼───────────────────────■───┤ Rz(3.2605) ├───┼────┼────────────────
      ├───────┤└─────────────┘      │        └───────────┘   │        │      ├───────────┴┐       │           ┌───┐       │   └────────────┘   │    │                
q_10: ┤ Ry(0) ├─────────────────────┼──────────────■─────────┼────────┼──────┤ Rz(3.6224) ├───────┼───────────┤ X ├───────┼────────────────────┼────┼────────────────
      ├───────┤                     │              │         │        │      └────────────┘       │           └─┬─┘       │                    │    │                
q_11: ┤ Ry(0) ├─────────────────────┼──────────────┼─────────┼────────┼───────────────────────────┼─────────────┼─────────┼────────────────────┼────┼────────────────
      ├───────┤ ┌─────

### AWS S3 Examples

In [27]:
aws_s3.upload_file(Filename='test.txt', 
                   Bucket='test-braket-bucket', 
                   Key='test-remote.txt')

In [28]:
aws_s3.download_file(Bucket='test-braket-bucket',
                     Key='test-remote.txt',
                     Filename='test-downloaded.txt')

In [30]:
with open('test-downloaded.txt') as file:

    print(file.read())

Test Task
